In [26]:
# https://qiita.com/makaishi2/items/63b7986f6da93dc55edd

url = 'https://www.aozora.gr.jp/cards/000148/files/794_ruby_4237.zip'
zip = '794_ruby_4237.zip'
import urllib.request
urllib.request.urlretrieve(url, zip)

('794_ruby_4237.zip', <http.client.HTTPMessage at 0x7fe79cba0ef0>)

In [27]:
import zipfile
with zipfile.ZipFile(zip, 'r') as myzip:
    myzip.extractall()
    # 解凍後のファイルからデータ読み込み
    for myfile in myzip.infolist():
        # 解凍後ファイル名取得
        filename = myfile.filename
        # ファイルオープン時にencodingを指定してsjisの変換をする
        with open(filename, encoding='sjis') as file:
            text = file.read()

In [28]:
import re
# ヘッダ部分の除去
text = re.split('\-{5,}',text)[2]
# フッタ部分の除去
text = re.split('底本：',text)[0]
# | の除去
text = text.replace('|', '')
# ルビの削除
text = re.sub('《.+?》', '', text)
# 入力注の削除
text = re.sub('［＃.+?］', '',text)
# 空行の削除
text = re.sub('\n\n', '\n', text) 
text = re.sub('\r', '', text)

# 整形結果確認

# 頭の100文字の表示 
print(text[:100])
# 見やすくするため、空行 
print()
print()
# 後ろの100文字の表示 
print(text[-100:])


一
　うとうととして目がさめると女はいつのまにか、隣のじいさんと話を始めている。このじいさんはたしかに前の前の駅から乗ったいなか者である。発車まぎわに頓狂な声を出して駆け込んで来て、いきなり肌をぬい


評に取りかかる。与次郎だけが三四郎のそばへ来た。
「どうだ森の女は」
「森の女という題が悪い」
「じゃ、なんとすればよいんだ」
　三四郎はなんとも答えなかった。ただ口の中で迷羊、迷羊と繰り返した。




In [29]:
!pip install janome | tail -n 1

In [30]:
# Janomeのロード
from janome.tokenizer import Tokenizer

# Tokenizerインスタンスの生成 
t = Tokenizer()

# テキストを引数として、形態素解析の結果、名詞・動詞・形容詞(原形)のみを配列で抽出する関数を定義 
def extract_words(text):
    tokens = t.tokenize(text)
    return [token.base_form for token in tokens 
        if token.part_of_speech.split(',')[0] in['名詞', '動詞']]

#  関数テスト
# ret = extract_words('三四郎は京都でちょっと用があって降りたついでに。')
# for word in ret:
#    print(word)

# 全体のテキストを句点('。')で区切った配列にする。 
sentences = text.split('。')
# それぞれの文章を単語リストに変換(処理に数分かかります)
word_list = [extract_words(sentence) for sentence in sentences]

# 結果の一部を確認 
for word in word_list[0]:
    print(word)

一
する
目
さめる
女
隣
じいさん
話
始める
いる


In [31]:
# Word2Vecライブラリの導入
!pip install gensim

In [32]:
# Word2Vecライブラリのロード
from gensim.models import word2vec

# size: 圧縮次元数
# min_count: 出現頻度の低いものをカットする
# window: 前後の単語を拾う際の窓の広さを決める
# iter: 機械学習の繰り返し回数(デフォルト:5)十分学習できていないときにこの値を調整する
# model.wv.most_similarの結果が1に近いものばかりで、model.dict['wv']のベクトル値が小さい値ばかりの 
# ときは、学習回数が少ないと考えられます。
# その場合、iterの値を大きくして、再度学習を行います。

# 事前準備したword_listを使ってWord2Vecの学習実施
model = word2vec.Word2Vec(word_list, vector_size=100, min_count=5, window=5)

In [33]:
# 結果の確認1
# 一つ一つの単語は100次元のベクトルになっています。 
# 「世間」のベクトル値を確認します。
print(model.__dict__['wv']['世間'])

[-0.04923886  0.07101192  0.00775438  0.036741   -0.02814334 -0.09849353
 -0.01104614  0.13414198 -0.02737278 -0.048366   -0.03012468 -0.06796414
  0.01120231  0.03806237  0.00602943 -0.02153015  0.017503   -0.06202307
  0.01568225 -0.13798797 -0.00231398  0.03966829  0.02469159 -0.0300553
 -0.02616142 -0.02341439 -0.06625162 -0.02770553 -0.03558978  0.03111415
  0.08412708  0.01671194  0.06029053 -0.07337933  0.00060224  0.08170196
  0.00106213 -0.04805532 -0.04920437 -0.09876283 -0.00700225 -0.03909159
 -0.04904477 -0.03318525  0.06357171 -0.02876736 -0.0406552  -0.00190353
  0.07041785  0.02837777  0.00499823 -0.01309842 -0.0655065   0.03500506
 -0.03606318  0.02963807  0.03933996 -0.03194635 -0.06423183  0.00175878
  0.03613277 -0.00743181 -0.01018242  0.01190657 -0.12025919  0.03375373
  0.01700754  0.01367206 -0.09538519  0.09154807 -0.04363927  0.01734594
  0.05013069 -0.00203852  0.0496249   0.01124255 -0.00343657 -0.04066565
 -0.05204115  0.01253024 -0.05009741 -0.01988001 -0.

In [35]:
# 結果の確認2
# 関数most_similarを使って「世間」の類似単語を調べます 
ret = model.wv.most_similar(positive=['男']) 
for item in ret:
    print(item[0], item[1])

まま 0.999679684638977
上 0.9996665120124817
前 0.9996634125709534
時 0.9996584057807922
ところ 0.9996504187583923
目 0.9996473789215088
女 0.999646008014679
出る 0.9996448159217834
自分 0.9996428489685059
頭 0.9996427297592163
